In [1]:
# Parameters
video_name = "Pierre_INM_3"  # default value for development

In [2]:
# General Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import gzip
from matplotlib import cm
import cv2
from ultralytics import YOLO
import yolox
from yolox.tracker.byte_tracker import BYTETracker
from yolox.tracker.byte_tracker import STrack
import torch
from scipy.spatial.transform import Rotation as R_scipy
from pykalman import KalmanFilter
import open3d as o3d


# Project Aria Imports
import projectaria_tools.core.mps as mps
from projectaria_tools.core import data_provider
from projectaria_tools.core.mps.utils import get_nearest_pose
from projectaria_tools.core.stream_id import StreamId
from projectaria_tools.core.calibration import (distort_by_calibration, get_linear_camera_calibration)
from projectaria_tools.core.sensor_data import TimeDomain, TimeQueryOptions

In [3]:
# # Input: Video (VRS file) + Slam Trajectories (With Aria SDK)

# closed_loop_path = "/Users/behnood/Downloads/S - Master Thesis - EPFL/Aria Recordings/Pierre/mps_Nas_InCM_65_vrs/slam/closed_loop_trajectory.csv"
# closed_loop_trajectory = mps.read_closed_loop_trajectory(closed_loop_path)

# vrs_file = "/Users/behnood/Downloads/S - Master Thesis - EPFL/Aria Recordings/Pierre/Nas_InCM_65.vrs"
# vrs_data_provider = data_provider.create_vrs_data_provider(vrs_file)

# semidense_points_path = "/Users/behnood/Downloads/S - Master Thesis - EPFL/Aria Recordings/Pierre/mps_Nas_InCM_65_vrs/slam/semidense_points.csv.gz"

# video_name = "Pierre_INM_1"
base_dir = "/Users/behnood/Downloads/S - Master Thesis - EPFL/Aria Recordings/Pierre"

# Build paths dynamically
vrs_file = os.path.join(base_dir, f"{video_name}.vrs")
closed_loop_path = os.path.join(base_dir, f"mps_{video_name}_vrs/slam/closed_loop_trajectory.csv")
semidense_points_path = os.path.join(base_dir, f"mps_{video_name}_vrs/slam/semidense_points.csv.gz")

# Load data
closed_loop_trajectory = mps.read_closed_loop_trajectory(closed_loop_path)
vrs_data_provider = data_provider.create_vrs_data_provider(vrs_file)

Loaded #closed loop trajectory poses records: 11301


[ProgressLogger][INFO]: 2025-07-23 18:51:20: Opening /Users/behnood/Downloads/S - Master Thesis - EPFL/Aria Recordings/Pierre/Pierre_INM_3.vrs...
[VRSIndexRecord][WARNING]: 1 record(s) not sorted properly. Sorting index.
[MultiRecordFileReader][DEBUG]: Opened file '/Users/behnood/Downloads/S - Master Thesis - EPFL/Aria Recordings/Pierre/Pierre_INM_3.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 231-1/mic activated
[VrsDataProvider][INFO]: streamId 247-1/baro0 activated
[VrsDataProvider][INFO]: streamId 281-1/gps activated
[VrsDataProvider][INFO]: streamId 282-1/wps activated
[VrsDataProvider][INFO]: streamId 283-1/bluetooth activated
[VrsDataProvider][WARNING]: Unsupported TimeSync mode: APP, ignoring.
[VrsDataProvider][INFO]: Fail to activate streamId 286-1
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]

In [4]:
# Load YOLO Model (use "yolov8n.pt" for speed, or "yolov8m.pt" for better accuracy)
model = YOLO("yolov8m.pt")

In [5]:
# Calculate RGB Camera Instrinsics

rgb_stream_id = StreamId("214-1")
rgb_stream_label = vrs_data_provider.get_label_from_stream_id(rgb_stream_id)
device_calibration = vrs_data_provider.get_device_calibration()
rgb_camera_calibration = device_calibration.get_camera_calib(rgb_stream_label)
print(rgb_camera_calibration)

CameraCalibration(label: camera-rgb, model name: Fisheye624, principal point: [710.741, 706.47], focal length: [611.091, 611.091], projection params: [611.091, 710.741, 706.47, 0.389076, -0.376021, -0.205402, 1.71844, -2.11538, 0.760929, -0.000219489, 0.000306255, 0.000164677, -0.000154579, -0.000721263, -0.000195109], image size (w,h): [1408, 1408], T_Device_Camera:(translation:[-0.00457615, -0.0118596, -0.0047664], quaternion(x,y,z,w):[0.329176, 0.0368128, 0.0306198, 0.943054]), serialNumber:0450577b730407504401100000000000, TimeOffsetSec_Device_Camera:0)


In [6]:
# Calculate RGB Camera Extrinsics

T_device_camera = rgb_camera_calibration.get_transform_device_camera()
principal_point = rgb_camera_calibration.get_principal_point()
projection_params = rgb_camera_calibration.get_projection_params()
focal_length = [projection_params[0], projection_params[0]]

T_device_camera.translation()  
translation = T_device_camera.translation()      
tx = translation[0][0]  
ty = translation[0][1]  
tz = translation[0][2]
t = [tx, ty, tz]   

rotation = T_device_camera.rotation()
quaternion = rotation.to_quat() 
R = R_scipy.from_quat(quaternion).as_matrix()

print("t: ", t, "\nR: ", R)

t:  [-0.004576149851854918, -0.01185964032054523, -0.004766398952129968] 
R:  [[[    0.78058     0.61861    0.089592]
  [    0.62312    -0.78141   -0.033516]
  [   0.049274    0.081988    -0.99541]]]


In [7]:
# Convert middle of bounding box to xyz in real world

def compute_3d_world_point(bbox, depth_map, principal_point, focal_length, R, t):
    x1, y1, x2, y2 = bbox
    u = int((x1 + x2) / 2)  
    v = int((y1 + y2) / 2)  

    return compute_3d_world_point_from_pixel(u, v, depth_map, principal_point, focal_length, R, t)

In [8]:
# Convert uv in picture, to xyz in real world

# First we use camera intrinsics to convert to xyz in camera world, 
# then we use R and t to convert to xyz in real world

def compute_3d_world_point_from_pixel(u, v, depth_map, principal_point, focal_length, R, t):
    d = depth_map[v, u]  
    
    fx, fy = focal_length  
    cx, cy = principal_point  

    # 3D point in camera coordinates
    x = (u - cx) / fx * d  # X coordinate in camera space
    y = (v - cy) / fy * d  # Y coordinate in camera space
    z = d  # Z coordinate is the depth

    # Camera coordinate (x, y, z)
    point_cam = np.array([[x], [y], [z]])

    # Convert rotation matrix and translation vector to numpy arrays
    R = np.array(R)
    R = np.squeeze(R)
    t = np.array(t).reshape(3, 1)

    # Transform to world coordinates
    point_world = R @ point_cam + t  # Apply rotation and translation

    # Return the 3D point in world coordinates
    xw, yw, zw = point_world.flatten() # 1D array [X, Y, Z]
    return [yw, xw, -zw]


In [9]:
# Convert xyz in real world to uvd in picture (d is for depth)

def project_3d_to_pixel_with_depth(world_point, principal_point, focal_length, R, t):
    # Unpack world point assuming it's in [y, x, -z] format
    y, x, minus_z = world_point
    z = -minus_z  # Restore original Z

    # Reconstruct world point in original [x, y, z] format
    point_world = np.array([[x], [y], [z]])

    # Convert R and t to appropriate numpy arrays
    R = np.squeeze(np.array(R))
    t = np.array(t).reshape(3, 1)

    # Invert extrinsic transformation: X_cam = R.T @ (X_world - t)
    point_cam = R.T @ (point_world - t)

    # Extract camera coordinates
    x_cam, y_cam, z_cam = point_cam.flatten()

    # Intrinsics
    fx, fy = focal_length
    cx, cy = principal_point

    # Project to pixel coordinates
    u = (x_cam * fx) / z_cam + cx
    v = (y_cam * fy) / z_cam + cy

    u = int(round(u))
    v = int(round(v))
    
    return u, v, z_cam

In [10]:
# Unzip and Visualize Semidense Point Cloud from Aria SDK

# Unzip (Load and display the first few rows)

with gzip.open(semidense_points_path, "rt") as f:
    df = pd.read_csv(f)

semidense_points = df[['px_world', 'py_world', 'pz_world']].to_numpy()
print("Loaded semidense point cloud:", semidense_points.shape)
dist_std = df['dist_std'].to_numpy()
semidense_points = semidense_points[dist_std <= 0.01]
print("High Q semidense point cloud:", semidense_points.shape)

# Visualize
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(semidense_points)

# o3d.visualization.draw_geometries([pcd])

Loaded semidense point cloud: (103695, 3)
High Q semidense point cloud: (41583, 3)


In [11]:
# Function for FaceBlur

# Skip the warning logs (Not working)
os.environ['GLOG_minloglevel'] = '2'  # Suppress INFO and WARNING logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # In case TensorFlow logging appears

import absl.logging
absl.logging.set_verbosity('error')  # Only show error logs, hide info/warnings

import mediapipe as mp

mp_face_detection = mp.solutions.face_detection


def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return enhanced
    

def blur_faces_in_box_mediapipe(frame, box):
    """
    Blurs faces detected within a given person bounding box in a video frame.

    Parameters:
        frame (np.ndarray): The full video frame (BGR).
        box (list): [x1, y1, x2, y2] bounding box coordinates for a person.
    """

    x1, y1, x2, y2 = box
    h, w = frame.shape[:2]

    # Add margin if needed
    margin = 0
    x1m = max(0, x1 - margin)
    y1m = max(0, y1 - margin)
    x2m = min(w, x2 + margin)
    y2m = min(h, y2 + margin)

    person_roi = frame[y1m:y2m, x1m:x2m]

    if person_roi.size == 0:
        print("empty ROI")
        return  # Empty ROI

    enhanced_roi = enhance_contrast(person_roi)

    # cv2.imshow("enhanced", enhanced_roi)

    # Mediapipe works with RGB images
    rgb_roi = cv2.cvtColor(enhanced_roi, cv2.COLOR_BGR2RGB)

    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.2) as face_detection:
        results = face_detection.process(rgb_roi)
        print("Results:", results)

        if not results.detections:
            print("no faces")
            return  # No faces detected

        for detection in results.detections:
            # Bounding box relative to the ROI
            bboxC = detection.location_data.relative_bounding_box
            x_rel = int(bboxC.xmin * person_roi.shape[1])
            y_rel = int(bboxC.ymin * person_roi.shape[0])
            w_rel = int(bboxC.width * person_roi.shape[1])
            h_rel = int(bboxC.height * person_roi.shape[0])

            # Coordinates relative to the full frame
            fx1 = x1m + x_rel
            fy1 = y1m + y_rel
            fx2 = fx1 + w_rel
            fy2 = fy1 + h_rel

            # Clip face coordinates to frame boundaries
            fx1 = max(0, fx1)
            fy1 = max(0, fy1)
            fx2 = min(w, fx2)
            fy2 = min(h, fy2)

            face_region = frame[fy1:fy2, fx1:fx2]
            if face_region.size == 0:
                continue

            blurred_face = cv2.GaussianBlur(face_region, (51, 51), 30)
            frame[fy1:fy2, fx1:fx2] = blurred_face


In [12]:
def init_trajectory_csv(csv_path):
    import os
    import csv

    if not os.path.exists(csv_path):
        with open(csv_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                "frame_index",
                "glasses_x", "glasses_y", "glasses_z",
                "opponent_x", "opponent_y", "opponent_z",
                "box_x1", "box_y1", "box_x2", "box_y2",
                "u", "v", "depth",
                "principal_x", "principal_y",
                "focal_x", "focal_y",
                "RR_00", "RR_01", "RR_02",
                "RR_10", "RR_11", "RR_12",
                "RR_20", "RR_21", "RR_22",
                "tt_x", "tt_y", "tt_z",
                "quat_w", "quat_x", "quat_y", "quat_z",
                "pedestrians_in_frame"
            ])


In [13]:
def write_trajectory_row(
    csv_path, frame_index, tt, object_3d_world_point, box,
    depth_map, principal_point, focal_length, RR, q_wxyz, pedestrians_in_frame
):
    import os
    import csv
    import json

    # Make sure CSV exists and header is written
    write_header = not os.path.exists(csv_path)

    u = (box[0] + box[2]) // 2
    v = (box[1] + box[3]) // 2
    depth_value = float(depth_map[v, u])

    quat_w, quat_x, quat_y, quat_z = q_wxyz

    pedestrians_serialized = json.dumps(pedestrians_in_frame)

    row = [
        frame_index,
        *tt,
        *object_3d_world_point,
        *box,
        u, v, depth_value,
        *principal_point,
        *focal_length,
        *RR.flatten(),
        *tt,  # duplicated for clarity
        quat_w, quat_x, quat_y, quat_z,
        pedestrians_serialized
    ]

    with open(csv_path, mode='a', newline='') as file:
        writer = csv.writer(file)

        if write_header:
            writer.writerow([
                "frame_index",
                "glasses_x", "glasses_y", "glasses_z",
                "opponent_x", "opponent_y", "opponent_z",
                "box_x1", "box_y1", "box_x2", "box_y2",
                "u", "v", "depth",
                "principal_x", "principal_y",
                "focal_x", "focal_y",
                "RR_00", "RR_01", "RR_02",
                "RR_10", "RR_11", "RR_12",
                "RR_20", "RR_21", "RR_22",
                "tt_x", "tt_y", "tt_z",
                "quat_w", "quat_x", "quat_y", "quat_z",
                "pedestrians_serialized"
            ])

        writer.writerow(row)


In [14]:
from transformers import pipeline
from PIL import Image
import matplotlib.pyplot as plt
import torch
import numpy as np
from transformers import DPTForDepthEstimation, DPTImageProcessor

model_id = "depth-anything/Depth-Anything-V2-Metric-Indoor-Base-hf"
pipe = pipeline("depth-estimation", model=model_id, device=0 if torch.cuda.is_available() else -1)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


In [15]:
def improve_depth_scale(depth_pred_map, principal_point, focal_length, R, t):
    image_width, image_height = depth_pred_map.shape[1], depth_pred_map.shape[0]
    
    with gzip.open(semidense_points_path, "rt") as f:
        df = pd.read_csv(f)
    
    semidense_points_world = df[['px_world', 'py_world', 'pz_world']].to_numpy()
    dist_std = df['dist_std'].to_numpy()
    
    # Filter for high-quality points
    semidense_points_world = semidense_points_world[dist_std <= 0.01]
    
    # === COMPARE DEPTHS ===
    depths_pred = []
    depths_real = []
    min_pred_depth = 1e-3
    
    for wp in semidense_points_world:
        try:
            u, v, z_real = project_3d_to_pixel_with_depth(
                world_point=wp,
                principal_point=principal_point,
                focal_length=focal_length,
                R=R,
                t=t
            )
    
            # Check bounds
            if 0 <= v < image_height and 0 <= u < image_width:
                z_pred = depth_pred_map[v, u]
                if z_pred > min_pred_depth and z_real > 0:
                    depths_pred.append(z_pred)
                    depths_real.append(z_real)
    
        except Exception as e:
            continue  # Skip invalid projections

    # === COMPUTE SCALE FACTOR ===
    depths_pred = np.array(depths_pred)
    depths_real = np.array(depths_real)
    
    if len(depths_pred) > 0:
        scale_factor = np.median(depths_real / depths_pred)
        print(f"Estimated scale factor: {scale_factor:.3f}")
        print(f"Valid depth correspondences: {len(depths_pred)}")
    
        # Apply scale to entire depth map
        depth_pred_scaled = depth_pred_map * scale_factor
        depth_pred_scaled = np.clip(depth_pred_scaled, 0.01, np.percentile(depth_pred_scaled, 99))
    else:
        print("No valid projections for scale correction.")
        scale_factor = 1.0
        depth_pred_scaled = depth_pred_map

    return depth_pred_scaled, scale_factor

In [16]:
# Apply Tracking, Depth, FaceBlur and Visualize the frames

import os

output_dir = os.path.join(base_dir, "Generated_Diagrams&Frames", video_name, "video_frames")
os.makedirs(output_dir, exist_ok=True)


W, H = 1408, 1408
focal = 450 * 1.48 
dst_calib = get_linear_camera_calibration(W, H, focal, "camera-rgb")
fx = fy = focal                       
cx = W / 2
cy = H / 2

focal_length   = [fx, fy]          
principal_point = [cy, cx]  
timestamps = vrs_data_provider.get_timestamps_ns(rgb_stream_id, TimeDomain.DEVICE_TIME)

glasses_xyz_positions = []
opponent_xyz_positions = []
pedestrians_xyz_positions = []

csv_path = os.path.join(base_dir, "Generated_Diagrams&Frames", video_name, "trajectory_data.csv")
if os.path.exists(csv_path):
    os.remove(csv_path)
init_trajectory_csv(csv_path)

count_shift = 0
sum_shift = 0.0

frame_index = -1

T_world_device = []

for timestamp in timestamps: # Selecting a portion of the frames in the video based on timestamp, [40:110] for Nima's Video in INM
    nearest_pose_info = get_nearest_pose(closed_loop_trajectory, timestamp)
    print(timestamp)
    if nearest_pose_info:
        T_world_device = nearest_pose_info.transform_world_device
    if T_world_device == []:
        continue

    frame_index += 1
    
    # Calculate Glasses XYZ
    translation = T_world_device.translation()      
    tx = translation[0][0]  
    ty = translation[0][1]  
    tz = translation[0][2]
    tt = [tx, ty, tz]   # Translation vector: position of glasses in world coordinates
    
    q_wxyz = rotation.to_quat().flatten() # Aria provides [w, x, y, z]
    q_xyzw = [q_wxyz[1], q_wxyz[2], q_wxyz[3], q_wxyz[0]]  # [x, y, z, w]
    RR = R_scipy.from_quat(q_xyzw).as_matrix()
    glasses_xyz_positions.append(tt)
    
    image_data, _ = vrs_data_provider.get_image_data_by_time_ns(
        rgb_stream_id, timestamp, TimeDomain.DEVICE_TIME, TimeQueryOptions.BEFORE
    )
    if image_data:
        img = image_data.to_numpy_array()  # Convert to NumPy array
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # Convert to OpenCV format

        # Apply the undistortion correction
        undistorted_rgb_image = distort_by_calibration(
            img_bgr, dst_calib, rgb_camera_calibration
        )

        frame = undistorted_rgb_image
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR (OpenCV) to RGB
        pil_frame = Image.fromarray(frame_rgb)  
        orig_w, orig_h = pil_frame.size
            
        depthResult = pipe(pil_frame)
        depthModel     = pipe.model
        processor = pipe.image_processor
        
        inputs = processor(images=pil_frame, return_tensors="pt").to(depthModel.device)
        with torch.no_grad():
            outputs = depthModel(**inputs)
            pred_depth = outputs.predicted_depth  
        
        small = pred_depth.squeeze(0).squeeze(0).cpu().numpy()  # shape: (h', w')
        depth_map = cv2.resize(small, (orig_w, orig_h), interpolation=cv2.INTER_CUBIC)
        
        depth_np = np.array(depth_map, dtype=np.float32) 

        print("▶️ Metric depth range:", depth_map.min(), "→", depth_map.max())

        results = model.track(
            source=frame, 
            persist=True, 
            tracker='/Users/behnood/miniforge3/lib/python3.10/site-packages/ultralytics/cfg/trackers/bytetrack.yaml'
        )
                
        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)  
        class_ids = results[0].boxes.cls  
        confidences = results[0].boxes.conf.cpu().numpy() 
        if results[0].boxes.id is not None:
            ids = results[0].boxes.id.cpu().numpy().astype(int)  # Get bounding box coordinates (x1, y1, x2, y2)
        else:
            ids = [404] * len(boxes)

        depth_np_normalized = (depth_np - depth_np.min()) / (depth_np.max() - depth_np.min())
        inv_depth_np_normalized = 1.0 - depth_np_normalized  # Invert the normalized values
        depth_colormap = cv2.applyColorMap((inv_depth_np_normalized * 255).astype(np.uint8), cv2.COLORMAP_INFERNO)
        # frame = depth_colormap
        
        current_id = 0
        pedestrians_in_frame = {}
        for box, id, class_id, confidence in zip(boxes, ids, class_ids, confidences):
            if class_id != 0:
                frame_filename = os.path.join(output_dir, f"frame_{frame_index:04d}.png")
                cv2.imwrite(frame_filename, frame)
                # print(frame_index, "@@@", len(opponent_xyz_positions))
                # cv2.imshow("Slam", frame)
                continue # Filter only the human objects
            # print("Person ", id, ", confidence: ", confidence)

            # object_3d_world_point = compute_3d_world_point(box, depth_map, principal_point, focal_length, RR, tt)

            current_id += 1
            
            if current_id > 7:
                continue # This line should be removed


            # NNNNEEEEWWWWW NEW
            scaled_depth_map, scale_val = improve_depth_scale(depth_map, principal_point, focal_length, RR, tt)
            print("Scale: ", scale_val)
            depth_map = scaled_depth_map

            
            object_3d_world_point = compute_3d_world_point(box, depth_map, principal_point, focal_length, RR, tt)

            opponent_xyz_positions.append(object_3d_world_point)
            
            print(frame_index, "!!!", len(opponent_xyz_positions))

            # write_trajectory_row(
            #     csv_path, frame_index, tt, object_3d_world_point, box,
            #     depth_map, principal_point, focal_length, RR, q_wxyz, 0
            # )

            # Draw bounding box normally (before rotation)
                
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 255), 2)

            # Apply FaceBlur
            blur_faces_in_box_mediapipe(frame, box)

            center_x = (box[0] + box[2]) // 2
            center_y = (box[1] + box[3]) // 2

            depth_value = float(depth_map[center_y, center_x])
            depth_value = round(depth_value, 2)            

            pedestrians_in_frame[int(id)] = {
                "xyz": object_3d_world_point,
                "depth": depth_value
            }
            # print(f"bbox: {box}\n")
            
            # Draw label in rotated-aware position
            # label = f"depth: {depth_value}"
            label = f"id: {id}, depth: {depth_value}m"
            (label_width, label_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            
            # Instead of box[1] - h (Y-axis), move on X-axis because of rotation
            label_x = box[1] - label_height - 5
            label_y = box[0]
            
            # Draw background rectangle for the label
            cv2.rectangle(
                frame,
                (label_y, label_x),  # note swapped coords
                (label_y + label_width, label_x + label_height + baseline),
                (0, 255, 255),
                thickness=cv2.FILLED
            )
            
            # Draw label text
            cv2.putText(
                frame,
                label,
                (label_y, label_x + label_height),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                thickness=1
            )

            # Show the frame
            frame_filename = os.path.join(output_dir, f"frame_{frame_index:04d}.png")
            cv2.imwrite(frame_filename, frame)

        if len(opponent_xyz_positions) <= frame_index:
            if len(boxes) == 0:
                frame_filename = os.path.join(output_dir, f"frame_{frame_index:04d}.png")
                cv2.imwrite(frame_filename, frame)   
            if len(opponent_xyz_positions) > 0:
                opponent_xyz_positions.append(opponent_xyz_positions[-1])
            else:
                opponent_xyz_positions.append([0, 0, 0])

        pedestrians_xyz_positions.append(pedestrians_in_frame)
        write_trajectory_row(
                csv_path, frame_index, tt, object_3d_world_point, box,
                depth_map, principal_point, focal_length, RR, q_wxyz, pedestrians_in_frame
            )
            # cv2.imshow("Slam", frame)

#             if cv2.waitKey(8) & 0xFF == ord('q'):
#                 break

# cv2.destroyAllWindows()

668400359962
668500356837
668600357462
668700356300
668800359800
668900354125
669000357462
669100358712
669200363050
669300356250
▶️ Metric depth range: 1.679136 → 19.979721

0: 640x640 4 persons, 1 remote, 168.8ms
Speed: 4.9ms preprocess, 168.8ms inference, 4.9ms postprocess per image at shape (1, 3, 640, 640)
Estimated scale factor: 0.325
Valid depth correspondences: 3900
Scale:  0.32488736178613153
0 !!! 1


I0000 00:00:1753289493.516232 5076740 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1753289493.525687 5077455 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Results: <class 'mediapipe.python.solution_base.SolutionOutputs'>
no faces
Estimated scale factor: 1.000
Valid depth correspondences: 3900
Scale:  0.9999999953763202
0 !!! 2
Results: <class 'mediapipe.python.solution_base.SolutionOutputs'>
no faces


I0000 00:00:1753289494.008016 5076740 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2 Pro
W0000 00:00:1753289494.009535 5077476 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Estimated scale factor: 1.000
Valid depth correspondences: 3900
Scale:  0.9999999953763202
0 !!! 3
Results: <class 'mediapipe.python.solution_base.SolutionOutputs'>
no faces


...

I0000 00:00:1753289856.198299 5076740 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2 Pro
W0000 00:00:1753289856.199357 5086866 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


▶️ Metric depth range: 1.6195697 → 12.223676

0: 640x640 1 person, 1 cell phone, 183.6ms
Speed: 2.7ms preprocess, 183.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Estimated scale factor: 0.603
Valid depth correspondences: 3023
Scale:  0.6033380462747984
111 !!! 328
Results: <class 'mediapipe.python.solution_base.SolutionOutputs'>
no faces
680500351625


I0000 00:00:1753289858.459497 5076740 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2 Pro
W0000 00:00:1753289858.460692 5086919 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


▶️ Metric depth range: 1.525147 → 12.583738

0: 640x640 1 person, 1 cell phone, 165.3ms
Speed: 2.2ms preprocess, 165.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Estimated scale factor: 0.387
Valid depth correspondences: 2915
Scale:  0.38713599633591567
112 !!! 329
Results: <class 'mediapipe.python.solution_base.SolutionOutputs'>
no faces


I0000 00:00:1753289860.659082 5076740 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2 Pro
W0000 00:00:1753289860.661076 5086942 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [ ]:
# from collections import defaultdict

# frame_counts = defaultdict(list)  # id -> list of frame indices

# # Build frame appearance index
# for frame_idx, id_dict in enumerate(pedestrians_xyz_positions):
#     for pid in id_dict:
#         frame_counts[pid].append(frame_idx)

# # Compute stable streaks
# def longest_consecutive_streak(frames):
#     streak = max_streak = 1
#     for i in range(1, len(frames)):
#         if frames[i] == frames[i - 1] + 1:
#             streak += 1
#             max_streak = max(max_streak, streak)
#         else:
#             streak = 1
#     return max_streak

# qualified_ids = []
# for pid, frames in frame_counts.items():
#     if len(frames) >= 15 and longest_consecutive_streak(frames) >= 15:
#         qualified_ids.append((pid, len(frames)))

# # Sort by appearance count, keep top 3
# qualified_ids.sort(key=lambda x: -x[1])
# top3_ids = [pid for pid, _ in qualified_ids[:3]]
# print("Top 3 stable IDs:", top3_ids)


In [17]:
from collections import defaultdict

# 1. Track frame appearances and depth coverage
frame_counts = defaultdict(list)            # id -> list of frame indices
depth_history = defaultdict(list)           # id -> list of depths

for frame_idx, id_dict in enumerate(pedestrians_xyz_positions):
    for pid, data in id_dict.items():
        frame_counts[pid].append(frame_idx)
        depth_history[pid].append(data["depth"])

# 2. Define utility for longest consecutive appearance streak
def longest_consecutive_streak(frames):
    if not frames:
        return 0
    frames.sort()
    streak = max_streak = 1
    for i in range(1, len(frames)):
        if frames[i] == frames[i - 1] + 1:
            streak += 1
            max_streak = max(max_streak, streak)
        else:
            streak = 1
    return max_streak

# 3. Filter candidates based on constraints
qualified_ids = []
for pid in frame_counts:
    appearances = frame_counts[pid]
    depths = depth_history[pid]

    if longest_consecutive_streak(appearances) >= 15 and any(d >= 2.0 for d in depths):
        qualified_ids.append((pid, len(appearances)))  # include total count for sorting

# 4. Sort by most frequent appearances
qualified_ids.sort(key=lambda x: -x[1])

# 5. Pick top 3 (or fewer if needed, but at least 1)
top_ids = [pid for pid, _ in qualified_ids[:3]]
if len(top_ids) == 0:
    # fallback: pick the best single ID based on raw appearance
    fallback = max(frame_counts.items(), key=lambda x: len(x[1]))[0]
    top_ids = [fallback]

print("✅ Selected top IDs for tracking:", top_ids)


✅ Selected top IDs for tracking: [2, 13, 12]


In [18]:
# Step 1: Initialize an empty list of length = total frames
num_frames = len(pedestrians_xyz_positions)
filtered_positions = [{} for _ in range(num_frames)]

# Step 2: Extract and complete each ID's timeline
for pid in top_ids:
    # Get frames where the ID is present
    pid_frames = [i for i, frame_data in enumerate(pedestrians_xyz_positions) if pid in frame_data]
    if not pid_frames:
        continue

    first_frame = pid_frames[0]
    last_frame = pid_frames[-1]

    # Get the first and last known values
    first_value = pedestrians_xyz_positions[first_frame][pid]
    last_value = pedestrians_xyz_positions[last_frame][pid]

    prev_value = None

    for i in range(num_frames):
        if pid in pedestrians_xyz_positions[i]:
            # Use actual data
            current_value = pedestrians_xyz_positions[i][pid]
            prev_value = current_value
        else:
            # Padding logic
            if i < first_frame:
                current_value = first_value  # Before first appearance
            elif i > last_frame:
                current_value = last_value  # After last appearance
            elif prev_value is not None:
                current_value = prev_value  # In-between gaps
            else:
                continue  # shouldn't happen, but safe fallback

        # Update filtered list
        filtered_positions[i][pid] = current_value

# print(filtered_positions)

In [ ]:
# filtered_positions = []
# for frame_data in pedestrians_xyz_positions:
#     filtered = {pid: frame_data[pid] for pid in top_ids if pid in frame_data}
#     filtered_positions.append(filtered)

# # print(filtered_positions)

In [ ]:
# # Function for Visualizing the xyz positions of the glasses and the pedestrian in 2D

# from IPython.display import HTML
# import matplotlib.animation as animation

# def visualize_two_3d_xyz(xyz_positions1, xyz_positions2):
#     xyz_positions1 = np.array(xyz_positions1)
#     xyz_positions2 = np.array(xyz_positions2)

#     fig, ax = plt.subplots()

#     # Extract coordinates
#     x1, y1 = xyz_positions1[:, 0], xyz_positions1[:, 1]
#     x2, y2 = xyz_positions2[:, 0], xyz_positions2[:, 1]

#     # Combine for axis scaling
#     all_x = np.concatenate((x1, x2))
#     all_y = np.concatenate((y1, y2))
    
#     x_min, x_max = np.min(all_x), np.max(all_x)
#     y_min, y_max = np.min(all_y), np.max(all_y)

#     scale_factor = 0.5
#     x_range = x_max - x_min
#     y_range = y_max - y_min
#     max_range = max(x_range, y_range)
#     x_center = (x_max + x_min) / 2
#     y_center = (y_max + y_min) / 2

#     ax.set_xlim(x_center - max_range / 2 - scale_factor, x_center + max_range / 2 + scale_factor)
#     ax.set_ylim(y_center - max_range / 2 - scale_factor, y_center + max_range / 2 + scale_factor)
#     ax.set_aspect('equal')

#     ax.set_xlabel("X Position (m)")
#     ax.set_ylabel("Y Position (m)")

#     # Two scatter plots for two sets of data
#     sc1 = ax.scatter([], [], c='red', label='Person 1')
#     sc2 = ax.scatter([], [], c='blue', label='Person 2')

#     ax.legend()

#     num_frames = 100
#     interval = 20000 / num_frames

#     def update(frame):
#         n1 = int((frame / num_frames) * len(x1))
#         n2 = int((frame / num_frames) * len(x2))

#         sc1.set_offsets(np.c_[x1[:n1], y1[:n1]])
#         sc2.set_offsets(np.c_[x2[:n2], y2[:n2]])
#         return sc1, sc2

#     ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=interval, blit=False)
#     plt.close(fig)
#     return HTML(ani.to_jshtml())


In [ ]:
# Function for smoothing trajectories with Kalman Filter (x and y are affected)

def smooth_xy_coordinates_with_kalman(data_xyz):
    """
    Apply Kalman filter to smooth x and y coordinates from 3D trajectory data.
    
    Parameters:
    data_xyz (list of [x, y, z]): Input list of 3D points.

    Returns:
    list of [x, y, z]: Smoothed 3D points, with x and y filtered using Kalman Filter.
    """
    data_xyz = np.array(data_xyz)
    observed_states = data_xyz[:, :2]  # Only x and y

    if len(observed_states) < 2:
        return data_xyz.tolist()

    # Kalman Filter setup
    transition_matrix = [[1, 1, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 1],
                         [0, 0, 0, 1]]
    
    observation_matrix = [[1, 0, 0, 0],
                          [0, 0, 1, 0]]
    
    initial_state_mean = [observed_states[0][0], 0, observed_states[0][1], 0]

    kf = KalmanFilter(
        transition_matrices=transition_matrix,
        observation_matrices=observation_matrix,
        transition_covariance=1e-5 * np.eye(4),
        observation_covariance=0.05**2 * np.eye(2),
        initial_state_mean=initial_state_mean
    )

    kf = kf.em(observed_states, n_iter=10)
    smoothed_states, _ = kf.smooth(observed_states)

    # Replace x and y with smoothed values, keep z the same
    smoothed_data = []
    for i, original in enumerate(data_xyz):
        smoothed_data.append([smoothed_states[i][0], smoothed_states[i][2], original[2]])

    return smoothed_data


In [ ]:
# # Step 1: Apply Kalman smoothing for each ID
# for pid in top_ids:
#     # Extract the xyz trajectory
#     trajectory_xyz = []
#     for frame in filtered_positions:
#         if pid in frame:
#             trajectory_xyz.append(frame[pid]["xyz"])
#         else:
#             trajectory_xyz.append(None)

#     # Fill in None values with the last known position to match what Kalman needs
#     filled_xyz = []
#     last_valid = trajectory_xyz[0]
#     for pt in trajectory_xyz:
#         if pt is None:
#             filled_xyz.append(last_valid)
#         else:
#             filled_xyz.append(pt)
#             last_valid = pt

#     # Apply Kalman smoothing
#     smoothed_xyz = smooth_xy_coordinates_with_kalman(filled_xyz)

#     # Update the filtered_positions with smoothed values
#     for i, frame in enumerate(filtered_positions):
#         if pid in frame:
#             frame[pid]["xyz"] = smoothed_xyz[i]

# print(filtered_positions)

In [ ]:
# # Smoothen and Visualize the trajectories of the glasses and the pedestrian

# # ONLY LINE THAT MATTERS
# smoothed_opponent_xyz_positions = smooth_xy_coordinates_with_kalman(opponent_xyz_positions)
# ## FOR TEST AND DEBUG
# # smoothed_opponent_xyz_positions = opponent_xyz_positions

# # visualize_two_3d_xyz(smoothed_opponent_xyz_positions, glasses_xyz_positions)
# # visualize_two_3d_xyz(opponent_xyz_positions, glasses_xyz_positions)

In [19]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np
import io
import base64
import matplotlib.animation as animation
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

def interactive_trajectory_view(xyz_positions1, xyz_positions2):
    xyz_positions1 = np.array(xyz_positions1)
    xyz_positions2 = np.array(xyz_positions2)

    x1, y1 = xyz_positions1[:, 0], xyz_positions1[:, 1]
    x2, y2 = xyz_positions2[:, 0], xyz_positions2[:, 1]

    all_x = np.concatenate((x1, x2))
    all_y = np.concatenate((y1, y2))

    x_min, x_max = np.min(all_x), np.max(all_x)
    y_min, y_max = np.min(all_y), np.max(all_y)

    scale_factor = 0.5
    max_range = max(x_max - x_min, y_max - y_min)
    x_center = (x_max + x_min) / 2
    y_center = (y_max + y_min) / 2

    def plot_frame(frame_idx):
        fig, ax = plt.subplots(figsize=(6, 6))

        ax.set_xlim(x_center - max_range / 2 - scale_factor, x_center + max_range / 2 + scale_factor)
        ax.set_ylim(y_center - max_range / 2 - scale_factor, y_center + max_range / 2 + scale_factor)
        ax.set_aspect('equal')
        ax.set_xlabel("X Position (m)")
        ax.set_ylabel("Y Position (m)")

        # Plot all positions up to the current frame
        ax.plot(x1[:frame_idx+1], y1[:frame_idx+1], c='red', label='Person 1')
        ax.plot(x2[:frame_idx+1], y2[:frame_idx+1], c='blue', label='Person 2')

        # Optionally, mark the current position with a larger dot
        ax.scatter([x1[frame_idx]], [y1[frame_idx]], c='red', s=40)
        ax.scatter([x2[frame_idx]], [y2[frame_idx]], c='blue', s=40)

        ax.legend()
        plt.show()

    interact(plot_frame, frame_idx=widgets.IntSlider(min=0, max=min(len(x1), len(x2)) - 1, step=1, value=0, description="Frame"))

    # Export HTML with interactive slider
    def export_interactive_html(output_path="trajectory_slider.html", frame_folder="frames"):
        os.makedirs(frame_folder, exist_ok=True)

        frame_paths = []
        for frame_idx in range(min(len(x1), len(x2))):
            fig, ax = plt.subplots(figsize=(6, 6))
            ax.set_xlim(x_center - max_range / 2 - scale_factor, x_center + max_range / 2 + scale_factor)
            ax.set_ylim(y_center - max_range / 2 - scale_factor, y_center + max_range / 2 + scale_factor)
            ax.set_aspect('equal')
            ax.set_xlabel("X Position (m)")
            ax.set_ylabel("Y Position (m)")
            ax.plot(x1[:frame_idx+1], y1[:frame_idx+1], c='red', label='Person 1')
            ax.plot(x2[:frame_idx+1], y2[:frame_idx+1], c='blue', label='Person 2')
            ax.scatter([x1[frame_idx]], [y1[frame_idx]], c='red', s=40)
            ax.scatter([x2[frame_idx]], [y2[frame_idx]], c='blue', s=40)
            ax.legend()

            frame_path = os.path.join(frame_folder, f"frame_{frame_idx:04d}.png")
            fig.savefig(frame_path, dpi=100)
            plt.close(fig)
            frame_paths.append(frame_path)

        # Write HTML with JS slider
        with open(output_path, "w") as f:
            f.write(f"""<!DOCTYPE html>
<html>
<head>
    <title>Trajectory Slider</title>
    <style>
        body {{ font-family: sans-serif; text-align: center; }}
        img {{ max-width: 100%; height: auto; }}
        #slider-container {{ margin: 20px; }}
        #frameLabel {{ margin-top: 10px; font-weight: bold; }}
        .image-row {{ display: flex; justify-content: center; gap: 20px; flex-wrap: wrap; }}
        .image-container {{ flex: 1 1 45%; }}
    </style>
</head>
<body>
    <h2>Trajectory Viewer</h2>
    <div id="slider-container">
        <input type="range" min="0" max="{len(frame_paths)-1}" value="0" id="frameSlider">
        <input type="number" min="0" max="{len(frame_paths)-1}" value="0" id="frameInput">
        <div id="frameLabel">Frame: 0</div>
    </div>
    <div class="image-row">
        <div class="image-container">
            <h3>Trajectory Plot</h3>
            <img id="trajectoryImg" src="diagram_frames/frame_0000.png" width="600">
        </div>
        <div class="image-container">
            <h3>Video Frame</h3>
            <img id="videoImg" src="video_frames/frame_0000.png" width="600">
        </div>
    </div>
    <script>
        const slider = document.getElementById('frameSlider');
        const input = document.getElementById('frameInput');
        const trajImg = document.getElementById('trajectoryImg');
        const videoImg = document.getElementById('videoImg');
        const label = document.getElementById('frameLabel');

        function updateFrame(val) {{
            const frameNum = String(val).padStart(4, '0');
            trajImg.src = 'diagram_frames/frame_' + frameNum + '.png';
            videoImg.src = 'video_frames/frame_' + frameNum + '.png';
            label.textContent = 'Frame: ' + val;
            slider.value = val;
            input.value = val;
        }}

        slider.oninput = function() {{ updateFrame(this.value); }}
        input.onchange = function() {{ updateFrame(this.value); }}
    </script>
</body>
</html>""")
#         with open(output_path, "w") as f:
#             f.write("""<!DOCTYPE html>
# <html>
# <head>
#     <title>Trajectory Slider</title>
#     <style>
#         body {{ font-family: sans-serif; text-align: center; }}
#         img {{ max-width: 100%; height: auto; }}
#         #slider-container {{ margin: 20px; }}
#         #frameLabel {{ margin-top: 10px; font-weight: bold; }}
#     </style>
# </head>
# <body>
#     <h2>Trajectory Viewer</h2>
#     <div id=\"slider-container\">
#         <input type=\"range\" min=\"0\" max=\"{max_frame}\" value=\"0\" id=\"frameSlider\">
#         <input type=\"number\" min=\"0\" max=\"{max_frame}\" value=\"0\" id=\"frameInput\">
#         <div id=\"frameLabel\">Frame: 0</div>
#     </div>
#     <div>
#         <img id=\"frameImg\" src=\"{folder}/frame_0000.png\" width=\"600\">
#     </div>
#     <script>
#         const slider = document.getElementById('frameSlider');
#         const input = document.getElementById('frameInput');
#         const img = document.getElementById('frameImg');
#         const label = document.getElementById('frameLabel');

#         function updateFrame(val) {{
#             const frameNum = String(val).padStart(4, '0');
#             img.src = '{folder}/frame_' + frameNum + '.png';
#             label.textContent = 'Frame: ' + val;
#             slider.value = val;
#             input.value = val;
#         }}

#         slider.oninput = function() {{ updateFrame(this.value); }}
#         input.onchange = function() {{ updateFrame(this.value); }}
#     </script>
# </body>
# </html>""".format(max_frame=len(frame_paths)-1, folder=frame_folder))

        print(f"✅ HTML + PNGs exported to: {output_path} and {frame_folder}/")

    frame_folder = os.path.join(base_dir, "Generated_Diagrams&Frames", video_name, "diagram_frames")
    # relative_frame_folder = os.path.basename(frame_folder)
    output_path = os.path.join(base_dir, "Generated_Diagrams&Frames", video_name, f"{video_name}_trajectory_slider.html")


    export_interactive_html(output_path=output_path, frame_folder=frame_folder)


In [ ]:
interactive_trajectory_view(glasses_xyz_positions, smoothed_opponent_xyz_positions)

In [ ]:
# def clean_xyz_sequence(xyz_seq):
#     return [
#         np.array(pt["xyz"], dtype=np.float32) if pt is not None else np.array([0, 0, 0], dtype=np.float32)
#         for pt in xyz_seq
#     ]

# person_id = top_ids[0]

# trajectory_xyz = []
# for frame_data in filtered_positions:
#     trajectory_xyz.append(frame_data.get(person_id, None))

# trajectory_xyz = clean_xyz_sequence(trajectory_xyz)

# # Now visualize
# interactive_trajectory_view(glasses_xyz_positions, trajectory_xyz)


In [20]:
person_id = top_ids[2]
print(person_id)

# Extract their already-smoothed, fully-padded xyz trajectory
trajectory_xyz = [frame[person_id]["xyz"] for frame in filtered_positions]

# Visualize against glasses trajectory
interactive_trajectory_view(glasses_xyz_positions, trajectory_xyz)

12


interactive(children=(IntSlider(value=0, description='Frame', max=112), Output()), _dom_classes=('widget-inter…

✅ HTML + PNGs exported to: /Users/behnood/Downloads/S - Master Thesis - EPFL/Aria Recordings/Pierre/Generated_Diagrams&Frames/Pierre_INM_3/Pierre_INM_3_trajectory_slider.html and /Users/behnood/Downloads/S - Master Thesis - EPFL/Aria Recordings/Pierre/Generated_Diagrams&Frames/Pierre_INM_3/diagram_frames/


In [21]:
print(len(smoothed_opponent_xyz_positions))
print(len(glasses_xyz_positions))
print(len(opponent_xyz_positions))
print(len(pedestrians_xyz_positions))

NameError: name 'smoothed_opponent_xyz_positions' is not defined